In [5]:
"""
Imports
"""

import cv2
import numpy as np
import pyAprilTag as pat


In [6]:
"""
Find April tags in images to get point correspondences. 
I will only be using the center cooordinates as point correspondences between images.
There are 40 April Tags in these images, so I will get 40 points in each image.

Here
- Id represents the type of April Tag
- corners, centers, and homographies can be understood from the names.
"""

image1 = cv2.imread("AprilCalib_orgframe_00007.png")
ids1, corners1, centers1, homographies1 = pat.find(image1)

image2 = cv2.imread("AprilCalib_orgframe_00008.png")
ids2, corners2, centers2, homographies2 = pat.find(image2)

print(ids1)
print(ids2)

[ 1  0  2  3  5  7  4  6  9  8 10 12 13 14 11 15 17 16 20 18 19 23 21 22
 25 26 24 30 29 28 27 31 38 39 37 36 33 35 32 34]
[ 0  1  2  3  5  6  4  7  8 10 11  9 13 12 14 15 17 16 18 19 20 21 22 23
 24 25 26 31 27 29 30 28 32 33 34 37 36 35 39 38]


In [7]:
"""
The point correspondences aren't in the right order. 
So first we need to correctly store them in the right order.
"""

# create 2 dictionaries to store corner points as per their id
points1 = dict(sorted(zip(ids1, centers1)))
points2 = dict(sorted(zip(ids2, centers2)))

print(f"Points in image 1: {list(points1.items())[0:3]}", end="\n\n")
print(f"Corresponding points in image 2: {list(points2.items())[:3]}", end="\n\n")


Few points in image 1: [(0, array([171.98628517,  37.64735978])), (1, array([238.21221728,  34.45608431])), (2, array([306.21845087,  33.90663495]))]

Corresponding points in image 2: [(0, array([ 80.50497922, 166.19489962])), (1, array([133.07805728, 173.07990659])), (2, array([186.9876852 , 180.60721997]))]



In [8]:
"""
Estimate fundamental matrix between two images
"""


def compute_fundamental_matrix(points1, points2):
    """
    Compute fundamental matrix between two images
    :param points1: dictionary of points in image 1
    :param points2: dictionary of points in image 2
    :return: fundamental matrix
    """

    # convert to numpy array
    points1 = np.array(list(points1.values()))
    points2 = np.array(list(points2.values()))

    # estimate fundamental matrix
    F, mask = cv2.findFundamentalMat(points1, points2, cv2.FM_LMEDS)

    # return fundamental matrix
    return F


F = compute_fundamental_matrix(points1, points2)
print(F)


[[-4.50756704e-06 -5.47177607e-05  3.26190789e-02]
 [ 4.95879263e-05 -9.38841117e-06  7.19771172e-03]
 [-3.38434320e-02 -3.46699368e-03  1.00000000e+00]]


In [9]:
def draw_epipolar_lines(img1, img2, lines, pts1, pts2):

    r, c = img1.shape
    img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)

    for r, pt1, pt2 in zip(lines, pts1, pts2):

        color = tuple(np.random.randint(0, 255, 3).tolist())

        x0, y0 = map(int, [0, -r[2] / r[1]])
        x1, y1 = map(int, [c, -(r[2] + r[0] * c) / r[1]])

        img1 = cv2.line(img1, (x0, y0), (x1, y1), color, 1)
        img1 = cv2.circle(img1, tuple(pt1), 5, color, -1)
        img2 = cv2.circle(img2, tuple(pt2), 5, color, -1)
    return img1, img2


In [10]:
img1_lines, img2_lines = draw_epipolar_lines(image1, image2, points1, points2, F)

cv2.imshow(img1_lines)
cv2.imshow(img2_lines)

cv2.waitKey(0)
cv2.destroyAllWindows()


ValueError: too many values to unpack (expected 2)

In [15]:
K = np.array(
    [
        [605.5268838, 0, 315.56751193],
        [0, 606.58216672, 255.62059543],
        [
            0,
            0,
            1,
        ],
    ]
)
essential_matrix = K.T @ F @ K


In [17]:
R1, R2, t = cv2.decomposeEssentialMat(essential_matrix)

In [19]:
print(R1)
print(R2)
print(t)

[[ 0.99039797 -0.10391459 -0.091179  ]
 [ 0.11315053  0.9882484   0.10277184]
 [ 0.07942801 -0.11210198  0.99051721]]
[[-0.49080152 -0.38874562 -0.77973759]
 [-0.52842439 -0.57874008  0.62115021]
 [-0.69273482  0.71689383  0.07862384]]
[[-0.55919728]
 [ 0.46481517]
 [ 0.68647305]]


In [20]:
essential_matrix

array([[ -1.65275718, -20.09793249,  10.42090611],
       [ 18.21373504,  -3.45439008,  12.40228755],
       [-13.6789604 , -14.03267945,   0.09114844]])